# Googler State Machine

This is a single-purpose state machine. It is characterized by a single line connecting all the states.

## State 0. Init

The starting point is to create the string representation of the following mermaid diagram as follows:

1. Add **CRAFT_TOOL_PROMPT**
2. Add **TOOL_CALL** transition to **GOOGLE: next / on_GOOGLE**.
3. Add **googler** to tools_dict

In [9]:
state_diagram = """stateDiagram-v2
    INIT --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT
        
    CRAFT_TOOL_PROMPT --> TOOL_CALL: next / on_TOOL_CALL        
        TOOL_CALL --> GOOGLE: next / on_GOOGLE
        GOOGLE --> GENERATE: next / on_GENERATE

    GENERATE --> PROCESS: next / on_PROCESS
        PROCESS --> END : next
    """

```mermaid
stateDiagram-v2
    INIT --> CRAFT_TOOL_PROMPT: next / on_CRAFT_PROMPT
        
    CRAFT_TOOL_PROMPT --> TOOL_CALL: next / on_TOOL_CALL        
        TOOL_CALL --> GOOGLE: next / on_GOOGLE
        GOOGLE --> GENERATE: next / on_GENERATE

    GENERATE --> PROCESS: next / on_PROCESS
        PROCESS --> END : next
```

In [10]:
# ARRANGE
import os
os.environ["LOG_LEVEL"]="WARNING"
#state_diagram_path = os.path.dirname(os.path.abspath(__name__)) + "/simple_agent_state_diagram.md"

from gai.ttt.client.ttt_client import TTTClient
ttt = TTTClient({
    "type": "ttt",
    "url": "http://localhost:12031/gen/v1/chat/completions"
})

from gai.persona.profile.pydantic.AgentPydantic import AgentPydantic
agent_data = AgentPydantic(
    Id="",
    Name="Alfred",
    AgentDescription="Hello, I am Alfred, the best large language model in the world, and I am here to assist you in any way possible. As a highly advanced AI assistant, I possess the ability to perform general-purpose tasks powered by <span ${style}>GPT-4</span>. With my vast knowledge base and powerful processing capabilities, I am able to provide you with the most relevant and helpful information available. Whether you need answers to complex questions, recommendations for products or services, or assistance with decision making, I am here to help. So, how may I be of service to you today?",
    AgentTraits="Wise,Serious,Meticulous",
    ImageUrl="",
    ThumbnailUrl=""
    )

tools_config={
    "google": {
        "tool_prompt":"👩‍🔬, use only the information provided to you by the user to answer the user''s question.\n            👩‍🔬, whenever possible, do not simply answer the question but try to be as informative as you can.\n            Remember, these information are scraped from the web so you may need to proofread and edit the content before responding.\n            👩‍🔬 will reply in point forms, precede each point with a newline \"\n\", and be precise in your articulation.\n            👩‍🔬 will provide your own reasoned subjective perspective, noting where your view differs from or expands on the contents.\n            Rules:\n                - Consolidate the materials provided by the user and then organise them point by point.\n                - Don't just answer the question, be as informative as you can. For example, provide and proofread some background information or fun-fact to support your answer and make it interesting.\n                - Begin your report by saying `According to my online research,...`\n                - Always provide your answers in point form.",
        "schema": {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "arguments": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }                    
        }
    }
}

This is a single action state machine

In [11]:
from gai.persona.fsm.AgentStateMachine import AgentStateMachine
agent = AgentStateMachine(
    ttt=ttt,
    rag=None,
    agent_data=agent_data,
    collection_name="demo",
    dialogue_messages=[],
    user_message="What is the current time in Singapore?",
    tools_config=tools_config,
    state_diagram=state_diagram,
    tool_name="google"
    ).Init()


Machine validation successful.


## State 2. CRAFT_TOOL_PROMPT

In [12]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:"+agent.state)


before:INIT
content: {'system_message': ''}
after:CRAFT_TOOL_PROMPT


## State 3. TOOL_CALL

In [13]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:"+agent.state)

before:CRAFT_TOOL_PROMPT
content: {"type": "function", "name": "google", "arguments": "{\"search_query\": \"current time Singapore\"}"}
after:TOOL_CALL


## State 4. GOOGLE

In [14]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:"+agent.state)

before:TOOL_CALL
content: [{'subject': 'current time Singapore', 'topic': "Sun & Moon Sun & Moon Today Sunrise & Sunset Moonrise & Moonset Moon Phases Eclipses Night Sky 15:26:19 SGT Monday, 21 October 2024 Fullscreen Country: Singapore Lat/Long: 1°17'N / 103°51'E Elevation: 93 m Currency: Singapore Dollar (SGD) Languages: English, Malay, Tamil, … Country Code: +65 °C Weather 32 °C Partly sunny. 33 / 25 °C Tue 22. 33 / 25 °C Wed 23. 32 / 24 °C Weather by CustomWeather, © 2024 More weather details Time Zone SGT (Singapore Time) UTC/GMT +8 hours No DST No Daylight Saving Time in 2024 Difference Same time as Singapore About SGT — Singapore Time Set your location Sunrise 06:46 ↑ 101° East Sunset 18:51 ↑ 259° West Day length 12 hours, 5 minutes -3s shorter Moon 80.2% Set – 10:12 Rise – 22:39 High Tide High – 01:47 High – 12:48 Low Tide Low – 07:14 Low – 19:40 More Sun & Moon in Singapore + Show More Twilight and Moon Phase Information Solar Noon Sun in South: 12:49 Altitude: 77.9° Astronomi

## State 5. GENERATE

In [7]:
print("before:"+agent.state)
agent.next()
print("after:",agent.state)
for chunk in agent.streamer:
    print(chunk,end="",flush=True)

before:GOOGLE
after: GENERATE
 According to my online research, the current time in Singapore is 14:16. This information can be found in the context provided, which includes details about the local time in Singapore. Singapore uses the Singapore Time (SGT) zone, which is UTC +8 hours. Additionally, the context also provides information about the time zone in Singapore, the local weather, and upcoming holidays in Singapore. The time in Singapore is on Monday, 21 October 2024, at 14:16

## State 6. PROCESS

In [8]:
print("before:"+agent.state)
agent.next()
print("content:",agent.content)
print("after:",agent.state)

before:GENERATE
content:  According to my online research, the current time in Singapore is 14:40 Singapore Standard Time (SGT), which is UTC +8. This means that it is 14 hours, 40 minutes ahead of Coordinated Universal Time (UTC). The day is Monday, 23rd September 2024. You can check the current time in other locations around the world using various tools and converters available online. Additionally, I found information about upcoming holidays in Singapore, such as Diwali/Deepavali on 31st October and Christmas Eve on 24th December
after: PROCESS
